###### Voronoi constructor Module 

### Usefull Libraries

In [6]:
import math
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.path as path

from scipy.spatial import Voronoi

from shapely.geometry import LineString, Polygon, MultiPolygon, Point
from shapely.ops import polygonize, unary_union

import import_ipynb
import node_linker
import pps
import utils

### Voronoi Diagram Generator

`def vor_prox(robots, vor, verbose)`

Generate the main Voronoi Diagram approximating the values of infinity vertices  
and returning the updated values of edges and vertices for BVC constructor.

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **vor&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;class scipy.spatial.Voronoi**
    * Voronoi diagram in N dimension
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices with approximation of infinity vertices
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge

In [40]:
def vor_prox(robots, vor, verbose):
    count = 0
    # Create copies of:
    vertices = vor.vertices
    edges = []
    # Find the center of the robots position
    center = robots.mean(axis=0)
    # pointidx: Indices of the points between which each Voronoi ridge lies.
    # simplex: Indices of the Voronoi vertices forming each Voronoi ridge.
    # zip join each values in a tuple
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        # Transform to array
        simplex_arr = np.array(simplex)
        # If the Voronoi edge goes to infinity
        if np.all(simplex_arr >= 0):
            # Append to the BVC edges list
            edges.append(simplex)
        # If one of the vertices goes to the infinity
        else:
            # Get the finite vertex
            i = simplex_arr[simplex_arr >= 0][0] 
            # Calculate the straight line between robots
            t = robots[pointidx[1]] - robots[pointidx[0]] 
            # Normalization of this line
            t = t / np.linalg.norm(t)
            # Ortogonalization
            n = np.array([-t[1], t[0]])
            # Get the midpoint between robots
            midpoint = robots[pointidx].mean(axis=0)
            # Far point = extension from the finite vertex using the
            #             orthogonalization and
            #             depending of the midpoint location respect center
            far_point = vor.vertices[i] + (np.sign(np.dot(midpoint- center,n))
                                           * n * 20 * len(robots))
            # Increase the count
            count += 1
            # Add the value to the BVC vertices array
            vertices = np.concatenate((vertices, far_point.reshape(1,2)),
                                      axis=0)
            # Update the pointer of BVC edges list 
            edges.append([len(vertices)-1, simplex[1]])
    if verbose:
        print('List of vertex coordinates (including approximations) of',
              'Voronoi Diagram:\n')
        for i in range(len(vertices)):
            print(i,".-",vertices[i])
        print('Robots in front: ',vor.ridge_points,'\n')
        print('Voronoi Diagram Edges: ',edges,'\n')
    return {"vrtx" : vertices, "edge" : edges}

### BVC Generator

`def bvc_gen(pos, radius, vor_prox, vrtx_assembled, robots_path, opt)`

Assemble the vertices for each cell in order to generate the new BVC vertices

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vor_prox&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Approximated values for Voronoi generation
* **vrtx_assembled&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of node_linker.DoublyLinkedList**
    * A list of ordered vertices for each robot
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarray**
    * Previous positions
* **opt&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
    
#### Return:
* **new_robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New position of robots

In [ ]:
def bvc_assembler(mode, milestone_node, vor_prox, radius, cell, bvc_vrtx, opt):
    # Auxiliar node for possible cuts
    aux_node = milestone_node.nref
    # Get coordinates
    vertex_start = vor_prox["vrtx"][milestone_node.item]
    vertex_mid1 = vor_prox["vrtx"][milestone_node.nref.item]
    vertex_mid2 = vor_prox["vrtx"][aux_node.item]
    vertex_end = vor_prox["vrtx"][aux_node.nref.item]
    # Get the new vertices in this sector
    sector = utils.intersect(vertex_start, vertex_mid1, vertex_end, radius)
    if mode == "open":
        # Assign the new values according special cases 
        if milestone_node is cell.start_node:
            bvc_vrtx.append(sector[0])
        bvc_vrtx.append(sector[1])
        if aux_node.nref.nref is None:
            bvc_vrtx.append(sector[2])
        
    elif mode == "closed":
        if opt["verbose"]:
            print(sector[1])
        bvc_vrtx.append(sector[1])
    return bvc_vrtx

In [2]:
def bvc_gen(pos, radius, vor_prox, robots_path, opt, vrtx_assembled):
    location = []
    set_of_new_vertices = [] # For storing the BVC vertices of each robot
    if opt["verbose"]:
        print("==================================================")
        print("                 BVC GENERATION")
        print("==================================================")
    # Calculate the bvc vertices and the closest point
    for index, cell in enumerate(vrtx_assembled):
        if opt["verbose"]:
            print('Analyzing these vertices...')
            cell.traverse_list()
        bvc_vrtx = []
        milestone_node = cell.start_node
        # Go in pairs of edges
        # Check if the cell is unlimited
        if cell.end_node.nref is None:
            if opt["verbose"]:
                print('\nOPEN CELL!')
                print('Number of vertices in this cell:',cell.lenght_list,'\n')
            while milestone_node.nref.nref is not cell.end_node.nref:
                bvc_vrtx = bvc_assembler("open", milestone_node, vor_prox,
                                         radius, cell, bvc_vrtx, opt)
                # Move the milestone_node 
                milestone_node = milestone_node.nref
            # Remove intersections of the BVC
            bvc_vrtx = utils.pruner(bvc_vrtx, opt["verbose"], "open")
        # If the cell is limited or closed
        else:
            if opt["verbose"]:
                print('\nCLOSED CELL!')
                print('Number of vertices in this cell:',cell.lenght_list)
            for _ in range(cell.lenght_list):
                bvc_vrtx = bvc_assembler("closed", milestone_node, vor_prox,
                                         radius, cell, bvc_vrtx, opt)
                # Move the milestone_node 
                milestone_node = milestone_node.nref
            # Remove intersections of the BVC
            ls = LineString(bvc_vrtx)
            lr = LineString(ls.coords[:] + ls.coords[0:1])
            if not lr.is_simple:
                if opt["verbose"]:
                    print("\nThere is an intersection!")
                mls = unary_union(lr)
                for polygon in polygonize(mls):
                    if polygon.contains(Point(pos["i"][index])):
                        break;
                bvc_vrtx = [np.array(k) for k in polygon.exterior.coords]
            else:
                bvc_vrtx.append(bvc_vrtx[0])
        # Save the new vertices
        set_of_new_vertices.append(bvc_vrtx)
        if opt["verbose"]:
            print('The new vertices are:',type(bvc_vrtx[0]))
            for i in range(len(bvc_vrtx)):
                print(bvc_vrtx[i])
            print('---------------------------------------------------------')
        # Check if target point is in/out of BVC
        bbPath = path.Path(np.array(bvc_vrtx))
        location.append(bbPath.contains_point(pos["f"][index]))
    return {"inside" : location, "vertices" : np.array(set_of_new_vertices)}